In [1]:
import os 
import re
import jieba
import pandas as pd

## 加载数据

In [2]:
data_in = "../data/百度题库/高中_历史/origin"

In [3]:
ancient_his_df = pd.read_csv(os.path.join(data_in, "古代史.csv"))

In [7]:
contemporary_his_df=pd.read_csv(os.path.join(data_in,'现代史.csv'))
modern_his_df=pd.read_csv(os.path.join(data_in,'近代史.csv'))

## 预处理

In [8]:
ancient_his_df.head()

,web-scraper-order,web-scraper-start-url,item
0,1566523436-2497,https://study.baidu.com/tiku,[题目]\n据《左传》记载，春秋后期鲁国大夫季孙氏的家臣阳虎独掌权柄后，标榜要替鲁国国君整肃...
1,1566523436-2506,https://study.baidu.com/tiku,[题目]\n秦始皇统一六国后创制了一套御玺。如任命国家官员，则封印“皇帝之玺”；若任命四夷的...
2,1566523436-2153,https://study.baidu.com/tiku,[题目]\n北宋加强中央集权的主要措施有（ ）\n①把主要将领的兵权收归中央②派文官担任...
3,1566523436-2328,https://study.baidu.com/tiku,[题目]\n商朝人崇信各种鬼神，把占卜、祭祀作为与神灵沟通的手段，负责通神事务的是商王和巫师...
4,1566523436-1914,https://study.baidu.com/tiku,[题目]\n公元963年，北宋政府在江淮地区设置了包括盐业管理，以及控制对茶叶销售的专卖等为...


In [9]:
def segment_line(line):
    line = re.sub(
            "[a-zA-Z0-9]|[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】《》“”！，。？、~@#￥%……&*（）]+|题目", '',line)
    tokens = jieba.cut(line, cut_all=False)
    
    return " ".join(tokens)

In [10]:
segment_line('[题目]\n据《左传》记载，春秋后期鲁国大夫季孙氏的家臣阳虎独掌权柄后，标榜要替鲁国国君整')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.658 seconds.
Prefix dict has been built successfully.


'据 左传 记载 春秋 后期 鲁国 大夫 季孙氏 的 家臣 阳虎 独掌 权柄 后 标榜 要 替 鲁国 国君 整'

In [12]:
%%time
ancient_his_df["item"] = ancient_his_df["item"].apply(lambda x: segment_line(x))
contemporary_his_df["item"] = contemporary_his_df["item"].apply(lambda x: segment_line(x))
modern_his_df["item"] = modern_his_df["item"].apply(lambda x: segment_line(x))

CPU times: user 4.08 s, sys: 22.4 ms, total: 4.11 s
Wall time: 4.11 s


In [13]:
ancient_his_df.head()

,web-scraper-order,web-scraper-start-url,item
0,1566523436-2497,https://study.baidu.com/tiku,据 左传 记载 春秋 后期 鲁国 大夫 季孙氏 的 家臣 阳虎 独掌 权柄 后 标榜 要 替...
1,1566523436-2506,https://study.baidu.com/tiku,秦始皇 统一 六国后 创制 了 一套 御玺 如 任命 国家 官员 则 封印 皇帝 之玺 ； ...
2,1566523436-2153,https://study.baidu.com/tiku,北宋 加强 中央集权 的 主要 措施 有 ① 把 主要 将领 的 兵权 收归 中央 ② 派 ...
3,1566523436-2328,https://study.baidu.com/tiku,商朝人 崇信 各种 鬼神 把 占卜 祭祀 作为 与 神灵 沟通 的 手段 负责 通神 事务 ...
4,1566523436-1914,https://study.baidu.com/tiku,公元 年 北宋 政府 在 江淮地区 设置 了 包括 盐业 管理 以及 控制 对 茶叶 销售 ...


## 贴标签

In [14]:
ancient_his_df["label"] = 0
contemporary_his_df["label"] = 1
modern_his_df["label"] = 2

In [21]:
modern_his_df.describe()

,label
count,1640.0
mean,2.0
std,0.0
min,2.0
25%,2.0
50%,2.0
75%,2.0
max,2.0


## 数据集

In [17]:
dataset_df = pd.concat([ancient_his_df, contemporary_his_df, modern_his_df])

In [23]:
print(ancient_his_df.describe())
print(contemporary_his_df.describe())
print(modern_his_df.describe())
dataset_df.describe()

        label
count  1000.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
        label
count  2330.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
        label
count  1640.0
mean      2.0
std       0.0
min       2.0
25%       2.0
50%       2.0
75%       2.0
max       2.0


,label
count,4970.000000
mean,1.128773
std,0.717432
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


## 提取特征

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

In [25]:
corpus = dataset_df['item']

$词频(TF)=某个词在章中出现的概率$

$词频(TF)=\left(\frac{某个词在文章中出现的概率}{文章总词数}\right)$

$IDF=\log \left(\frac{语料库的文档总数}{包含该词的文档数+1}\right)$

$TF-IDF=TF*IDF$
> TF-IDF与一个词在文档中的出现次数成正比，与该词在整个语言中的出现次数成反比。所以，自动提取关键词的算法就很清楚了，就是计算出文档的每个词的TF-IDF值，然后按降序排列，取排在最前面的几个词。

In [26]:
vectorizer = TfidfVectorizer(max_features=2500, min_df=5)

In [27]:
X = vectorizer.fit_transform(corpus)

In [29]:
print(vectorizer.get_feature_names())
print(len(vectorizer.get_feature_names()))

['一个', '一五', '一些', '一人', '一位', '一体', '一体化', '一切', '一国两制', '一场', '一天', '一定', '一带', '一年', '一律', '一战', '一批', '一方面', '一时期', '一条', '一样', '一次', '一点', '一直', '一种', '一系列', '一致', '一般', '一词', '一起', '一边倒', '一部', '一部分', '一项', '万元', '万有引力', '万物', '万隆会议', '三个', '三个代表', '三公九卿', '三司', '三国', '三大', '三年', '三权分立', '三次', '三民主义', '三省', '三种', '三纲五常', '三项', '上升', '上帝', '上海', '上述', '下列', '下图', '下层', '下表', '下表为', '下表是', '下议院', '下降', '下面', '不仅', '不会', '不再', '不利于', '不可', '不合', '不同', '不够', '不得', '不得不', '不敢', '不断', '不断加强', '不断扩大', '不断进步', '不是', '不治', '不满', '不知', '不符', '不结盟', '不结盟运动', '不能', '不许', '不足', '不过', '不选', '不难', '与题', '专制', '专制主义', '专制制度', '专政', '专门', '世界', '世界各地', '世界市场', '世界贸易组织', '世界银行', '世纪', '世纪末', '世袭', '世袭制', '世贸组织', '东南', '东方', '东欧', '东汉', '东西', '东西方', '丞相', '两个', '两代', '两党制', '两国', '两大', '两岸', '两弹一星', '两极', '两次', '两种', '两者', '两项', '严刑峻法', '严密', '严峻形势', '严格', '严禁', '严重', '丧失', '个人', '个体', '个性', '中世纪', '中书', '中书省', '中体西用', '中共', '中共中央', '中共十一届三中全会', '中华', '中华人民共和国', '中华民国', '中华民族', '中叶', '中后期', '中国', '

In [30]:
print(X.shape)

(4970, 2500)


In [32]:
X[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

## Naive Bayes

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X.toarray(), dataset_df['label'], test_size=0.2, random_state=42)

In [37]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB,BernoulliNB

### 1. GaussianNB（高斯贝叶斯）

In [40]:
%%time
# 构建模型
gnb = GaussianNB()

# 训练
gnb = gnb.fit(X_train, y_train)

# 预测
gnb_y_pred = gnb.predict(X_test)

CPU times: user 151 ms, sys: 69.7 ms, total: 220 ms
Wall time: 248 ms


### 2. MultinomialNB （多项分布朴素贝叶斯）

In [43]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3976, 2500)
(994, 2500)
(3976,)
(994,)


In [44]:
%%time
# 构建模型

clf = MultinomialNB()
# 训练

clf = clf.fit(X_train, y_train)

# 预测
clf_y_pred = clf.predict(X_test)

CPU times: user 364 ms, sys: 477 ms, total: 841 ms
Wall time: 46.4 ms


### 3. ComplementNB（补充朴素贝叶斯）

In [45]:
%%time
# 构建模型
cnb = ComplementNB()

# 训练
cnb = cnb.fit(X_train, y_train)

# 预测
cnb_y_pred = cnb.predict(X_test)

CPU times: user 667 ms, sys: 661 ms, total: 1.33 s
Wall time: 54.2 ms


### 4. BernoulliNB（伯努利朴素贝叶斯）

In [46]:
%%time
# 构建模型
bnb = BernoulliNB()

# 训练
bnb = bnb.fit(X_train, y_train)

# 预测
bnb_y_pred = bnb.predict(X_test)

CPU times: user 967 ms, sys: 1.05 s, total: 2.02 s
Wall time: 138 ms


## 评估

In [47]:
from sklearn.metrics import classification_report

In [48]:
print('GauusianNB classification_report: \n')
print(classification_report(y_test, gnb_y_pred))
print('MultinomialNB classification_report: \n')
print(classification_report(y_test, clf_y_pred))
print('ComplementNB classification_report: \n')
print(classification_report(y_test, cnb_y_pred))
print('BernoulliNB classification_report: \n')
print(classification_report(y_test, bnb_y_pred))

GauusianNB classification_report: 

              precision    recall  f1-score   support

           0       0.68      0.83      0.75       213
           1       0.70      0.47      0.56       451
           2       0.58      0.76      0.66       330

    accuracy                           0.64       994
   macro avg       0.65      0.69      0.65       994
weighted avg       0.66      0.64      0.63       994

MultinomialNB classification_report: 

              precision    recall  f1-score   support

           0       0.94      0.89      0.91       213
           1       0.77      0.83      0.80       451
           2       0.76      0.71      0.74       330

    accuracy                           0.80       994
   macro avg       0.82      0.81      0.82       994
weighted avg       0.80      0.80      0.80       994

ComplementNB classification_report: 

              precision    recall  f1-score   support

           0       0.85      0.92      0.89       213
           1    

In [49]:
from sklearn.metrics import confusion_matrix

In [53]:
help(confusion_matrix)

Help on function confusion_matrix in module sklearn.metrics._classification:

confusion_matrix(y_true, y_pred, labels=None, sample_weight=None, normalize=None)
    Compute confusion matrix to evaluate the accuracy of a classification.
    
    By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
    is equal to the number of observations known to be in group :math:`i` and
    predicted to be in group :math:`j`.
    
    Thus in binary classification, the count of true negatives is
    :math:`C_{0,0}`, false negatives is :math:`C_{1,0}`, true positives is
    :math:`C_{1,1}` and false positives is :math:`C_{0,1}`.
    
    Read more in the :ref:`User Guide <confusion_matrix>`.
    
    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.
    
    y_pred : array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    
    labels : array-like of shape (n_classes), default=Non

In [56]:
print('GauusianNB confusion_matrix: \n')
print(confusion_matrix(y_test, gnb_y_pred))
print('MultinomialNB confusion_matrix: \n')
print(confusion_matrix(y_test, clf_y_pred))
print('ComplementNB confusion_matrix: \n')
print(confusion_matrix(y_test, cnb_y_pred))
print('BernoulliNB confusion_matrix: \n')
print(confusion_matrix(y_test, bnb_y_pred))

GauusianNB confusion_matrix: 

[[177  28   8]
 [ 65 210 176]
 [ 19  60 251]]
MultinomialNB confusion_matrix: 

[[189  23   1]
 [  5 374  72]
 [  7  88 235]]
ComplementNB confusion_matrix: 

[[197  13   3]
 [ 24 344  83]
 [ 11  73 246]]
BernoulliNB confusion_matrix: 

[[184  23   6]
 [  3 361  87]
 [  4  74 252]]
